In [30]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append("../")

location = "remote"
if location == "remote":
    # TODO: hacky, shouldn't be necessary
    os.environ["WANDB_NOTEBOOK_NAME"] = "lustre_scratch/coralshift/notebooks/gru.ipynb"
    os.chdir("/lustre_scratch/orlando-code/coralshift/")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from __future__ import annotations

from pathlib import Path
import xarray as xa
import numpy as np
# import math as m
# import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
# import matplotlib.patches as patches

import wandb
from tqdm import tqdm
from sklearn import model_selection
# from sklearn.preprocessing import normalize
from scipy.interpolate import interp2d
from sklearn.utils import class_weight
# from scipy.ndimage import gaussian_gradient_magnitude
import xbatcher


from tensorflow.keras import layers
import keras
# import rasterio
# from rasterio.plot import show
# import rioxarray as rio

from coralshift.processing import spatial_data
from coralshift.utils import file_ops, directories
from coralshift.plotting import spatial_plots, model_results
from coralshift.dataloading import data_structure, climate_data

In [32]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        print(tf.config.experimental.get_visible_devices('GPU'))
    except RuntimeError as e:
        print(e)

2 Physical GPUs, 1 Logical GPUs
[PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [33]:
# gpu_options.allow_growth = True
!nvidia-smi

Sun Jun 25 01:02:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:86:00.0 Off |                    0 |
| N/A   32C    P0    46W / 163W |  31460MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:89:00.0 Off |                    0 |
| N/A   

In [34]:
cells = 12800
seq_len = 10000
num_fs = 15
target_frac = 0.1

# Creating the features array
features = np.random.random((cells, seq_len, num_fs)).astype(np.float32)

# Creating the label array
labels = np.random.choice((2), size=(cells,), p=[1-target_frac,target_frac])

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)

# Verifying the shape of the arrays
print("Features shape:", features.shape)
print("Label shape:", labels.shape)
print("Class weights:", class_weights)

Features shape: (12800, 10000, 15)
Label shape: (12800,)
Class weights: [0.55507372 5.03937008]


In [ ]:
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

In [35]:
# send numpy features/label array to tf.Data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((features, labels))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 10

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
print("Number of batches:", len(train_dataset))

Number of batches: 400


## Replacing toys with real data

In [45]:
# TODO: replace with all variables
dir = directories.get_datasets_dir() / "test"
Xs, ys = np.load(dir / "Xs_lstm.npy"), np.load(dir / "ys_lstm.npy")

Xs = np.moveaxis(Xs,1,2)

print("Xs shape: ", Xs.shape)
print("ys shape: ", ys.shape)



Xs shape:  (25619, 336, 4)
ys shape:  (25619,)


In [46]:
# X_train = Xs[:1000, :, :20]
# y_train = ys[:1000]
X_train = Xs
y_train = ys


class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
print("class weights: ", class_weights)



class weights:  [0.54231583 6.40795398]


In [47]:
# send numpy features/label array to tf.Data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 10

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
print("Number of batches:", len(train_dataset))

Number of batches: 401


In [ ]:
# wandb.init(
#     project="coralshift",
#     entity="orlando-code",
#     settings=wandb.Settings(start_method="fork")
#     # config={    }
#     )

# # initialize optimiser: will need hyperparameter scan for learning rate and others
# # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
# optimizer = tf.keras.optimizers.Adam(3e-4)

# # X = ds_man.get_dataset("monthly_climate_1_12_X_np")
# # y = ds_man.get_dataset("monthly_climate_1_12_y_np")
# # # check that untrained model runs (should output array of non-nan values)
# # # why values change?
# # # g_model(X[:32])

# # X_train, X_test, y_train, y_test = model_selection.train_test_split(
# #     X, y, test_size=0.2, random_state=42)

# # X_train, X_test, y_train, y_test = model_selection.train_test_split(
# #     sub_X, sub_y, test_size=0.2, random_state=42)

# # Define Gated Recurrent Unit model class in TensorFlow
# class gru_model(tf.keras.Model):
#     # initialise class instance to define layers of the model
#     def __init__(self, rnn_units: list[int], num_layers: int, 
#         # dff: int
#         ):
#         """Sets up a GRU model architecture with multiple layers and dense layers for mapping the outputs of the GRU 
#         layers to a desired output shape

#         Parameters
#         ----------
#         rnn_units (list[int]): list containing the number of neurons to use in each layer
#         num_layers (int): number of layers in GRU model
#         """
#         super(gru_model, self).__init__()   # initialise GRU model as subclass of tf.keras.Model
#         # store values for later use
#         self.num_layers = num_layers    # number of layers in GRU model
#         self.rnn_units = rnn_units
#         # self.dff = dff
#         # define model layers: creating new `tf.keras.layers.GRU` layer for each iteration
#         self.grus = [tf.keras.layers.GRU(rnn_units[i],  # number (integer) of rnn units/neurons to use in each model layer
#                                    return_sequences=True,   # return full sequence of outputs for each timestep
#                                    return_state=True) for i in range(num_layers)] # return last hidden state of RNN at end of sequence
        
#         # dense layers are linear mappings of RNN layer outputs to desired output shape
#         # self.w1 = tf.keras.layers.Dense(dff) # 10 units
#         self.w1 = tf.keras.layers.Dense(10) # 10 units

#         self.w2 = tf.keras.layers.Dense(1)  # 1 unit (dimension 1 required before final sigmoid function)
#         # self.A = tf.keras.layers.Dense(30)
#         # self.B = tf.keras.layers.Dense(dff)



#     def call(self, inputs: np.ndarray, training: bool=False):
#         """Processes an input sequence of data through several layers of GRU cells, followed by a couple of
#         fully-connected dense layers, and outputs the probability of an event happening.
        
#         Parameters
#         ----------
#         inputs (np.ndarray): input tensor of shape (batch_size, seq_length, features)
#             batch_size - defines the size of the sample drawn from datapoints
#             seq_length - number of timesteps in sequence
#             features - number of features associated with each datapoint
#         training (bool, defaults to False): True if model is in training, False if in inference mode

#         Returns
#         -------
#         target: probability of an event occuring, with shape (batch_size, 1)
#         """
#         # input shape: (batch_size, seq_length, features)
       
#         assert self.num_layers == len(self.rnn_units)

#         # check that input tensor has correct shape
#         if (len(inputs.shape) != 3):
#             print(f"Incorrect shape of input tensor. Expected 3D array. Recieved {len(inputs.shape)}D array.")

#         # print('input dim ({}, {}, {})'.format(inputs.shape[0], inputs.shape[1], inputs.shape[2]))
#         # whole_seq, static_input = inputs
#         whole_seq = inputs


#         # iteratively passes input tensor to GRU layers, overwriting preceding sequence 'whole_seq'
#         for layer_num in range(self.num_layers):
#             whole_seq, final_s = self.grus[layer_num](whole_seq, training=training)

#         # adding extra layers
#         # static = self.B(tf.nn.gelu(self.A(static_input)))
#         # target = self.w1(final_s)  + static # final hidden state of last layer used as input to fully connected dense layers...
#         target = self.w1(final_s)   # final hidden state of last layer used as input to fully connected dense layers...

#         target = tf.nn.relu(target) # via ReLU activation function
#         target = self.w2(target)    # final hidden layer must have dimension 1 
        
#         # obtain a probability value between 0 and 1
#         target = tf.nn.sigmoid(target)
        
#         return target


# # initialise GRU model with 500 hidden layers, one GRU unit per layer 
# g_model = gru_model([100], 1) # N.B. [x] is number of hidden layers in GRU network


# def negative_log_likelihood(y: np.ndarray, y_pred: np.ndarray, class_weights: np.ndarray = None) -> float:
#     """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
#     incorporating class weights.

#     Parameters
#     ----------
#     y (np.ndarray): true binary labels, where 0 represents the negative class
#     y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)
#     class_weights (np.ndarray): weights for each class. If None, no class weights will be applied.

#     Returns
#     -------
#     float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred',
#     incorporating class weights if provided
#     """
#     bce = tf.keras.losses.BinaryCrossentropy()  

#     if class_weights is not None:
#         sample_weights = tf.gather(class_weights, np.asarray(y,dtype=np.int32))
#         # reshape to match size of y and y_pred
#         return bce(y, y_pred, sample_weight=tf.reshape(sample_weights, (-1, 1)))

#     return bce(y, y_pred)


# def training_batches(X: np.ndarray, y: np.ndarray, batch_num: int, batch_size: int=32):
#     start_idx = batch_num * batch_size
#     end_idx = (batch_num + 1) * batch_size

#     X_batch = X[start_idx:end_idx]
#     y_batch = y[start_idx:end_idx]
    
#     return X_batch, y_batch

# # https://stackoverflow.com/questions/52357542/attributeerror-tensor-object-has-no-attribute-numpy
# # should aim to delete the following to speed up training: but can't figure out a way to make wandb reporting work
# # without it
# tf.config.run_functions_eagerly(True)

# def build_graph():
    
#     # compile function as graph using tf's autograph feature: leads to faster execution times, at expense of limitations
#     # to Python objects/certain control flow structures (somewhat relaxed by experimental_relax_shapes)
#     @tf.function(experimental_relax_shapes=True)
#     def train_step(gru: tf.keras.Model, optimizer: tf.keras.optimizers.Optimizer, X: tf.Tensor, y: tf.Tensor, 
#                num_batches: int, batch_num: int, training: bool = True, class_weights=class_weights
#                ) -> tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
#         """Train model using input `X` and target data `y` by computing gradients of the loss (via 
#         negative_log_likelihood)
        
#         Parameters
#         ----------
#         y (np.ndarray): true binary labels, where 0 represents the negative class
#         y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)

#         Returns
#         -------
#         float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred'
#         """
#         if training:

#             num_batches = len(X)
#             # num_samples = X.shape[0]
#             # num_batches = num_samples // batch_size
#             # num_batches = batch_num
#             # total_epoch_loss = 0.0
#             # for batch_num in tqdm(range(num_batches), desc="batches", position=0, leave=True):
#             # for batch_num, batch in tqdm(enumerate(ds), desc="batches", position=0, leave=True):
#             total_batch_loss = 0

#             y_pred = gru(X, training)
#             xent = negative_log_likelihood(y, y_pred, class_weights)

#                 # X_batch, y_batch = training_batches(X, y, batch_num=batch_num, batch_size=batch_size)

#             with tf.GradientTape(persistent=True) as tape:
#                 y_pred = gru(X, training) 
#                 xent = negative_log_likelihood(y, y_pred, class_weights)
#                 # print(xent)
#                 # print(y_pred)
            
#             gradients = tape.gradient(xent, gru.trainable_variables)
#             optimizer.apply_gradients(zip(gradients, gru.trainable_variables))
#             # print("xent", xent.numpy())
#             # print("total_epoch_loss", total_epoch_loss)
#             total_batch_loss += xent
#                 # learning rate?
#             wandb.log({"batch": batch_num, "loss": xent, "total_epoch_loss": total_batch_loss})

#             # average_loss = total_batch_loss / num_batches
#             # return predicted output values and total loss value
#             return y_pred, xent, total_batch_loss

#     # set default float type
#     tf.keras.backend.set_floatx('float32')
#     return train_step


# with tf.device("/GPU:1"):
#     num_epochs = 20
#     # will update so that subsamples are fed in from which batches are taken: will require recomputation
#     # of class_weight for each subsample

#     tr_step = build_graph()
#     num_batches = len(train_dataset)

#     for epoch in tqdm(range(num_epochs), desc= " epochs", position=1, leave=True):
#         ongoing_epoch_loss = 0.0
#         for i, (X_batch, y_batch) in tqdm(enumerate(train_dataset), position=0, total=len(train_dataset), desc=" training on batches", leave=True):
#             y_pred, xent, batch_loss = tr_step(
#                 g_model, optimizer, X_batch, y_batch, class_weights=class_weights, 
#                 num_batches = num_batches, batch_num=i,
#                 training=True)

#             ongoing_epoch_loss += batch_loss
#             # print(ongoing_epoch_loss)
#             wandb.log({"ongoing_epoch_loss": ongoing_epoch_loss})
#         average_epoch_loss = ongoing_epoch_loss / len(train_dataset)
#         # print(average_epoch_loss)
#         wandb.log({"epoch": epoch, "tota_epoch_loss": average_epoch_loss})

# wandb.finish()


In [ ]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [ ]:
class TransformerModel(tf.keras.Model):
    def __init__(
        self,
        model_input_shape,
        head_size,
        num_heads,
        ff_dim,
        num_transformer_blocks,
        mlp_units,
        dropout=0,
        mlp_dropout=0,
        n_classes=2,
    ):
        super(TransformerModel, self).__init__()

        self.model_input_shape = model_input_shape
        self.head_size = head_size
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_transformer_blocks = num_transformer_blocks
        self.mlp_units = mlp_units
        self.dropout = dropout
        self.mlp_dropout = mlp_dropout
        self.n_classes = n_classes

        self.build_model()

    def transformer_encoder(self, inputs):
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(
            key_dim=self.head_size, num_heads=self.num_heads, dropout=self.dropout
        )(x, x)
        x = layers.Dropout(self.dropout)(x)
        res = x + inputs

        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=self.ff_dim, kernel_size=1, activation="relu")(x)
        x = layers.Dropout(self.dropout)(x)
        x = layers.Conv1D(filters=self.model_input_shape[-1], kernel_size=1)(x)
        return x + res

    def build_model(self):
        inputs = keras.Input(shape=self.model_input_shape)
        x = inputs

        for _ in range(self.num_transformer_blocks):
            x = self.transformer_encoder(x)

        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in self.mlp_units:
            x = layers.Dense(dim, activation="relu")(x)
            x = layers.Dropout(self.mlp_dropout)(x)
        outputs = layers.Dense(self.n_classes, activation="softmax")(x)

        self.model = keras.Model(inputs, outputs)


In [ ]:

# configure model
LEARNING_RATE = 1e-3    # TODO: adjustable (not particularly preessing though)
# INPUT_SHAPE = (None, samples, seq_len, num_features)
INPUT_SHAPE =  Xs.shape[1:]
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
EPOCHS = 4
HEAD_SIZE = 256
NUM_HEADS = 4
FF_DIM = 4
NUM_TRANSFORMER_BLOCKS = 4
MLP_UNITS = 128
MLP_DROPOUT = 0.4
DROPOUT = 0.25
N_CLASSES = 2

# Create an instance of the TransformerModel
transformer_model = TransformerModel(
    model_input_shape=INPUT_SHAPE,
    head_size=HEAD_SIZE,
    num_heads=NUM_HEADS,
    ff_dim=FF_DIM,
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    mlp_units=[MLP_UNITS],
    dropout=DROPOUT,
    mlp_dropout=MLP_DROPOUT,
    n_classes=N_CLASSES,
)

# Access the underlying model
transformer_model.model.summary()


In [ ]:
transformer_model.model

In [ ]:
transformer_model.model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=["sparse_categorical_accuracy"],
)

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

transformer_model.model.fit(
    Xs[:1000],
    ys[:1000],
    validation_split=0.2,
    epochs=2,
    batch_size=64,
    callbacks=callbacks,
)

In [ ]:
####################################
# DEFINE TRANSFORMER MODEL
####################################


class CoralTransformer(tf.keras.Model):
    """Transformer-based model for binary classification of (spatio)temporal data"""

    # initialise class instance to define layers of the model
    def __init__(self, inputs, head_size, num_heads, ff_dim, dropout=0, epsilon=1e-6, activation="Relu", kernel_size=1):

        # initialise CoralTransformer model as subclass of tf.keras.Model
        super(CoralTransformer, self).__init__()

        # Normalization and Attention layers
        self.layer_norm = layers.LayerNormalization(
            epsilon=epsilon
            )
        self.multi_head_attention = layers.MultiHeadAttention(
            key_dim=head_size, num_heads=num_heads, dropout=dropout
            )
        self.dropout = layers.Dropout(
            dropout
            )

        # Feed Forward layers
        self.conv1D_0 = layers.Conv1D(
            filters=ff_dim, kernel_size=kernel_size, activation=activation
            )
        self.conv1D_1 = layers.Conv1D(
            filters=inputs.shape[-1], kernel_size=kernel_size
            )


    def transformer_encoder(self, inputs, training=False):
        # Normalizationi and Attention
        x = self.layer_norm(inputs)
        x = self.multi_head_attention(x, x)
        x = self.dropout(x)
        result = x + inputs

        # Feed Forward
        x = self.layer_norm(result)
        x = self.conv1D_0(x)
        x = self.dropout(x)
        x = self.conv1D_1(x)
        return x + result

        # x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        # x = layers.MultiHeadAttention(
        #     key_dim=head_size, num_heads=num_heads, dropout=dropout
        # )(x, x)
        # x = layers.Dropout(dropout)(x)
        # res = x + inputs

        # # Feed Forward Part
        # x = layers.LayerNormalization(epsilon=1e-6)(res)
        # x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
        # x = layers.Dropout(dropout)(x)
        # x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        # return x + res

    def build_model(
        input_shape,
        head_size,
        num_heads,
        ff_dim,
        num_transformer_blocks,
        mlp_units,
        dropout,
        mlp_dropout,
    ):
        inputs = keras.Input(shape=input_shape)
        x = inputs

        for _ in range(num_transformer_blocks):
            x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in mlp_units:
            x = layers.Dense(dim, activation="relu")(x)
            x = layers.Dropout(mlp_dropout)(x)
        outputs = layers.Dense(n_classes, activation="softmax")(x)
        return keras.Model(inputs, outputs)


###########################
# INITIALISE PARAMETERS
###########################


# configure model
LEARNING_RATE = 1e-3    # TODO: adjustable (not particularly preessing though)
# INPUT_SHAPE = (None, samples, seq_len, num_features)
INPUT_SHAPE =  Xs.shape[1:]
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
EPOCHS = 4
HEAD_SIZE = 256
NUM_HEADS = 4
FF_DIM = 4
NUM_TRANSFORMER_BLOCKS = 4
MLP_UNITS = 128
MLP_DROPOUT = 0.4
DROPOUT = 0.25

model(build_model)

model = build_model(
    input_shape,
    activation=ACTIVATION,
    head_size=HEAD_SIZE,
    num_heads=NUM_HEADS,
    ff_dim=FF_DIM,
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    mlp_units=[MLP_UNITS],
    dropout=DROPOUT,
    mlp_dropout=MLP_DROPOUT,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()



In [ ]:
class TransformerModel(tf.keras.Model):
    def __init__(
        self,
        model_input_shape,
        head_size,
        num_heads,
        ff_dim,
        num_transformer_blocks,
        mlp_units,
        dropout=0,
        mlp_dropout=0,
        n_classes=2,
    ):
        super(TransformerModel, self).__init__()

        self.model_input_shape = model_input_shape
        self.head_size = head_size
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_transformer_blocks = num_transformer_blocks
        self.mlp_units = mlp_units
        self.dropout = dropout
        self.mlp_dropout = mlp_dropout
        self.n_classes = n_classes

    def transformer_encoder(self, inputs, training=False):
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(
            key_dim=self.head_size, num_heads=self.num_heads, dropout=self.dropout
        )(x, x)
        x = layers.Dropout(self.dropout)(x)
        res = x + inputs

        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=self.ff_dim, kernel_size=1, activation="relu")(x)
        x = layers.Dropout(self.dropout)(x)
        x = layers.Conv1D(filters=self.model_input_shape[-1], kernel_size=1)(x)
        return x + res

    def call(self, inputs, training=False):
        inputs = keras.Input(shape=self.model_input_shape)
        x = inputs

        for _ in range(self.num_transformer_blocks):
            x = self.transformer_encoder(x, training=training)

        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in self.mlp_units:
            x = layers.Dense(dim, activation="relu")(x)
            x = layers.Dropout(self.mlp_dropout)(x, training=training)
        outputs = layers.Dense(self.n_classes, activation="softmax")(x)

        self.model = keras.Model(inputs, outputs)

    




In [27]:
class TransformerModel(tf.keras.Model):
    def __init__(
        self,
        model_input_shape,
        head_size,
        num_heads,
        ff_dim,
        num_transformer_blocks,
        mlp_units,
        dropout=0,
        mlp_dropout=0,
        n_classes=2,
    ):
        super(TransformerModel, self).__init__()

        self.model_input_shape = model_input_shape
        self.head_size = head_size
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_transformer_blocks = num_transformer_blocks
        self.mlp_units = mlp_units
        self.dropout = dropout
        self.mlp_dropout = mlp_dropout
        self.n_classes = n_classes

        self.transformer_layers = []
        for _ in range(self.num_transformer_blocks):
            self.transformer_layers.append(self.transformer_encoder())

        self.global_average_pooling = layers.GlobalAveragePooling1D(
            data_format="channels_first"
        )

        self.dense_layers = []
        for units in self.mlp_units:
            self.dense_layers.append(layers.Dense(units, activation="relu"))

        self.dropout_layers = []
        for _ in range(len(self.mlp_units)):
            self.dropout_layers.append(layers.Dropout(self.mlp_dropout))

        self.output_layer = layers.Dense(self.n_classes, activation="softmax")

    def transformer_encoder(self):
        def encoder(inputs, training=False):
            x = layers.LayerNormalization(epsilon=1e-6)(inputs)
            x = layers.MultiHeadAttention(
                key_dim=self.head_size, num_heads=self.num_heads, dropout=self.dropout
            )(x, x, training=training)
            x = layers.Dropout(self.dropout)(x, training=training)
            res = x + inputs

            x = layers.LayerNormalization(epsilon=1e-6)(res)
            x = layers.Conv1D(filters=self.ff_dim, kernel_size=1, activation="relu")(x)
            x = layers.Dropout(self.dropout)(x, training=training)
            x = layers.Conv1D(filters=self.model_input_shape[-1], kernel_size=1)(x)
            return x + res

        return encoder

    def call(self, inputs, training=False):

        x = inputs
        for i in range(self.num_transformer_blocks):
            x = self.transformer_layers[i](x, training=training)

        x = self.global_average_pooling(x)

        for i in range(len(self.dense_layers)):
            x = self.dense_layers[i](x)
            x = self.dropout_layers[i](x, training=training)

        outputs = self.output_layer(x)
        # self.model = keras.Model(inputs, outputs)

        return outputs

In [67]:
# # set up distributed processing
# mirrored_strategy = tf.distribute.MirroredStrategy()
# GLOBAL_BATCH_SIZE = (BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync)

# with mirrored_strategy.scope():
# transformer_model = TransformerModel(activation=wandb.config["activation"])

class TransformerModel(tf.keras.Model):
    def __init__(
        self,
        model_input_shape,
        head_size,
        num_heads,
        ff_dim,
        num_transformer_blocks,
        mlp_units,
        dropout=0,
        mlp_dropout=0,
        n_classes=2,
    ):
        super(TransformerModel, self).__init__()

        self.model_input_shape = model_input_shape
        self.head_size = head_size
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_transformer_blocks = num_transformer_blocks
        self.mlp_units = mlp_units
        self.dropout = dropout
        self.mlp_dropout = mlp_dropout
        self.n_classes = n_classes

        self.transformer_layers = []
        for _ in range(self.num_transformer_blocks):
            self.transformer_layers.append(self.transformer_encoder())

        self.global_average_pooling = layers.GlobalAveragePooling1D(
            data_format="channels_first"
        )

        self.dense_layers = []
        for units in self.mlp_units:
            self.dense_layers.append(layers.Dense(units, activation="relu"))

        self.dropout_layers = []
        for _ in range(len(self.mlp_units)):
            self.dropout_layers.append(layers.Dropout(self.mlp_dropout))

        self.output_layer = layers.Dense(1, activation="sigmoid")

    def transformer_encoder(self):
        def encoder(inputs, training=False):
            x = layers.LayerNormalization(epsilon=1e-6)(inputs)
            x = layers.MultiHeadAttention(
                key_dim=self.head_size, num_heads=self.num_heads, dropout=self.dropout
            )(x, x, training=training)
            x = layers.Dropout(self.dropout)(x, training=training)
            res = x + inputs

            x = layers.LayerNormalization(epsilon=1e-6)(res)
            x = layers.Conv1D(filters=self.ff_dim, kernel_size=1, activation="relu")(x)
            x = layers.Dropout(self.dropout)(x, training=training)
            x = layers.Conv1D(filters=self.model_input_shape[-1], kernel_size=1)(x)
            return x + res

        return encoder

    def call(self, inputs, training=False):

        x = inputs
        for i in range(self.num_transformer_blocks):
            x = self.transformer_layers[i](x, training=training)

        x = self.global_average_pooling(x)

        for i in range(len(self.dense_layers)):
            x = self.dense_layers[i](x)
            x = self.dropout_layers[i](x, training=training)

        outputs = self.output_layer(x)
        self.model = keras.Model(inputs, outputs)

        return outputs



# Create an instance of the TransformerModel
transformer_model = TransformerModel(
    model_input_shape=INPUT_SHAPE,
    head_size=HEAD_SIZE,
    num_heads=NUM_HEADS,
    ff_dim=FF_DIM,
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    mlp_units=[MLP_UNITS],
    dropout=DROPOUT,
    mlp_dropout=MLP_DROPOUT,
    n_classes=N_CLASSES,
)
DIST_OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)



def negative_log_likelihood(
    labels: np.ndarray, predictions: np.ndarray, class_weights: np.ndarray = None, distributed: bool=False) -> float:
    """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
    incorporating class weights.

    Parameters
    ----------
    y (np.ndarray): true binary labels, where 0 represents the negative class
    y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)
    class_weights (np.ndarray): weights for each class. If None, no class weights will be applied.

    Returns
    -------
    float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred',
    incorporating class weights if provided
    """
    bce = tf.keras.losses.BinaryCrossentropy()  

    if distributed:
        bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)  

    print("labels shape:", tf.shape(labels))
    print("predictions shape:", tf.shape(predictions))

    # TODO: this may be broken, throwing error associated with numpy/tensor incompatability
    if class_weights is not None:
        # sample_weights = tf.gather(class_weights, np.asarray(tf.concat(labels.values, axis=0), dtype=np.int32))
        sample_weights = tf.gather(class_weights, tf.cast(labels, dtype=tf.int32))
        sample_weights = tf.expand_dims(sample_weights, axis=1)  # Add a new dimension

        # reshape to match size of y and y_pred
        return bce(labels, predictions, sample_weight=tf.reshape(sample_weights, (-1, 1)))

    return bce(labels, predictions)


def compute_loss(
    labels: tf.Tensor, predictions: tf.Tensor, class_weights: tuple[float] = None, distributed: bool = False):
    """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
    incorporating class weights.

    Parameters
    ----------
        labels (tf.Tensor): True binary labels, where 0 represents the negative class.
        predictions (tf.Tensor): Predicted labels as probability values between 0 and 1.
        class_weights (np.ndarray): Weights for each class. If None, no class weights will be applied.
        distributed (bool): Flag indicating if the computation is distributed.

    Returns
    -------
        float: Negative log likelihood loss computed using binary cross-entropy loss between 'labels' and 
            'predictions', incorporating class weights if provided.
    """
    per_example_loss = negative_log_likelihood(labels, predictions, class_weights, distributed)
    return tf.nn.compute_average_loss(per_example_loss, global_batch_size=BATCH_SIZE)

train_accuracy = tf.keras.metrics.BinaryCrossentropy(name="train_accuracy")
test_accuracy = tf.keras.metrics.BinaryCrossentropy(name="test_accuracy")
# would use BinaryAccuracy if predicting ones/zeros


@tf.function
def train_step(model: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor, 
    optimizer: tf.keras.optimizers.Optimizer = OPTIMIZER, learn_rate: float = LEARNING_RATE, 
    class_weights: tuple = None, distributed: bool=False):

    # features = tf.keras.Input(tensor=features)

    with tf.GradientTape() as lstm_tape:
        predictions = model(features, training=True)
        # Calculate loss
        loss = compute_loss(labels, predictions, class_weights, distributed)
    
    # get trainable variables
    trainable_vars = model.trainable_variables
    # Calculate gradient          `
    lstm_grads = lstm_tape.gradient(loss, trainable_vars)
    # And then apply the gradient to change the weights
    optimizer.apply_gradients(zip(lstm_grads, trainable_vars))

    train_accuracy.update_state(labels, predictions)
    return loss


def test_step(model: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor):

    predictions = model(features, training=False)
    test_accuracy.update_state(labels, predictions)

def create_model():
    inputs = tf.keras.Input(shape=INPUT_SHAPE)
    transformer_output = transformer_model(inputs)  # Wrap transformer_model with InputLayer
    outputs = tf.keras.layers.Dense(1, activation='softmax')(transformer_output)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


model = create_model()


# for epoch in tqdm(range(EPOCHS), desc="epochs", position=0):
#     # Just total loss
#     epoch_loss = 0
#     # Iterate through the train dataset
#     for features, labels in tqdm(train_dataset, desc=" train batches", position=1):
#         loss = train_step(transformer_model, features, labels, DIST_OPTIMIZER, LEARNING_RATE,
#             class_weights
#             )
#         epoch_loss += loss
#         print(epoch_loss)
#         print(train_accuracy.result())
        # wandb.log({"epoch loss": epoch_loss, "train accuracy": train_accuracy.result()})

    # # Iterate through the test dataset
    # for features, labels in tqdm(dist_train_dataset, desc=" test batches", position=1):
    #     distributed_test_step(conv_lstm, features, labels)
    #     wandb.log({"test accuracy": test_accuracy.result()})



ValueError: Exception encountered when calling layer "transformer_model_25" (type TransformerModel).

in user code:

    File "/tmp/ipykernel_28444/452054373.py", line 81, in call  *
        self.model = keras.Model(inputs, outputs)
    File "/home/jovyan/lustre_scratch/conda-envs/coralshift/lib/python3.10/site-packages/keras/engine/functional.py", line 158, in __init__
        [
    File "/home/jovyan/lustre_scratch/conda-envs/coralshift/lib/python3.10/site-packages/keras/engine/functional.py", line 159, in <listcomp>
        functional_utils.is_input_keras_tensor(t)
    File "/home/jovyan/lustre_scratch/conda-envs/coralshift/lib/python3.10/site-packages/keras/engine/functional_utils.py", line 48, in is_input_keras_tensor
        raise ValueError(_KERAS_TENSOR_TYPE_CHECK_ERROR_MSG.format(tensor))

    ValueError: Found unexpected instance while processing input tensors for keras functional model. Expecting KerasTensor which is from tf.keras.Input() or output from keras layer call(). G

In [66]:
model.summary()

NameError: name 'model' is not defined

In [50]:
transformer_model.model.summary()

AttributeError: 'TransformerModel' object has no attribute 'model'

In [39]:
train_accuracy

In [ ]:
###########################
# INITIALISE PARAMETERS
###########################


# configure model
LEARNING_RATE = 1e-3    # TODO: adjustable (not particularly preessing though)
BATCH_SIZE = 32
# INPUT_SHAPE = (None, samples, seq_len, num_features)
MODEL_INPUT_SHAPE = Xs.shape[1:]
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
EPOCHS = 4
HEAD_SIZE = 256
NUM_HEADS = 4
FF_DIM = 4
NUM_TRANSFORMER_BLOCKS = 4
MLP_UNITS = 128
MLP_DROPOUT = 0.4
DROPOUT = 0.25

# Create an instance of the TransformerModel
transformer_model = TransformerModel(
    model_input_shape=MODEL_INPUT_SHAPE,
    head_size=HEAD_SIZE,
    num_heads=NUM_HEADS,
    ff_dim=FF_DIM,
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    mlp_units=[MLP_UNITS],
    dropout=DROPOUT,
    mlp_dropout=MLP_DROPOUT,
    n_classes=2,
)

# transformer_model = transformer_model(Xs[:10])

In [25]:
# configure model
LEARNING_RATE = 1e-3    # TODO: adjustable (not particularly preessing though)
# INPUT_SHAPE = (None, samples, seq_len, num_features)
INPUT_SHAPE =  Xs.shape[1:]
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
EPOCHS = 4
HEAD_SIZE = 256
NUM_HEADS = 4
FF_DIM = 4
NUM_TRANSFORMER_BLOCKS = 4
MLP_UNITS = 128
MLP_DROPOUT = 0.4
DROPOUT = 0.25
N_CLASSES = 2
BATCH_SIZE = 8

# Split the data into train and validation sets
train_size = int(0.8 * len(Xs[:1000]))
train_X, val_X = Xs[:train_size], Xs[train_size:]
train_y, val_y = ys[:train_size], ys[train_size:]

# Create a dataset from the training data
train_dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y)).batch(BATCH_SIZE)

In [26]:
optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# Create an instance of the TransformerModel
transformer_model = TransformerModel(
    model_input_shape=INPUT_SHAPE,
    head_size=HEAD_SIZE,
    num_heads=NUM_HEADS,
    ff_dim=FF_DIM,
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    mlp_units=[MLP_UNITS],
    dropout=DROPOUT,
    mlp_dropout=MLP_DROPOUT,
    n_classes=N_CLASSES,
)

transformer_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["sparse_categorical_accuracy"],
)

def train_step(model: tf.keras.Model, optimizer: tf.keras.optimizers.Optimizer, 
        X: np.ndarray, y: np.ndarray, 
        training: bool=True, 
        # class_weights=class_weights, 
        batch_num:int=None, batch_size: int=None):

        with tf.GradientTape() as tape:
            # Forward pass
            logits = transformer_model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)

        # Compute gradients
        grads = tape.gradient(loss_value, transformer_model.trainable_weights)

        # Update weights
        optimizer.apply_gradients(zip(grads, transformer_model.trainable_weights))

        # Update metrics
        train_loss_metric.update_state(loss_value)
        train_acc_metric.update_state(y_batch_train, logits)

        # Print training progress every 100 steps
        if step % 100 == 0:
            print(f"Step {step}/{len(train_X)//batch_size}, Loss: {train_loss_metric.result()}, Accuracy: {train_acc_metric.result()}")


        # inputs = keras.Input(shape=self.model_input_shape)

# Define the loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Define metrics
train_loss_metric = tf.keras.metrics.Mean()
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_loss_metric = tf.keras.metrics.Mean()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Custom training loop
for epoch in range(EPOCHS):
    # Reset metrics at the start of each epoch
    train_loss_metric.reset_states()
    train_acc_metric.reset_states()

    # Iterate over batches of the training dataset
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
       
        train_step(transformer_model, optimizer, 
            x_batch_train, y_batch_train, batch_num = step, batch_size=BATCH_SIZE)


    # Perform validation at the end of each epoch
    val_logits = transformer_model(val_X, training=False)
    val_loss_value = loss_fn(val_y, val_logits)

    # Update validation metrics
    val_loss_metric.update_state(val_loss_value)
    val_acc_metric.update_state(val_y, val_logits)

    # Print validation results
    print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss_metric.result()}, Validation Accuracy: {val_acc_metric.result()}")

    # Reset validation metrics for the next epoch
    val_loss_metric.reset_states()
    val_acc_metric.reset_states()

TypeError: TransformerModel.transformer_encoder() missing 1 required positional argument: 'inputs'

In [ ]:
transformer_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=["sparse_categorical_accuracy"],
)

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    Xs,
    ys,
    validation_split=0.2,
    epochs=2,
    batch_size=64,
    callbacks=callbacks,
)

In [ ]:
prediction = np.reshape(model.predict(Xs), (187,137,2))


In [ ]:
 def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


In [ ]:
input_shape = Xs.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()



In [ ]:
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

# model.fit(
#     Xs,
#     ys,
#     validation_split=0.2,
#     epochs=2,
#     batch_size=64,
#     callbacks=callbacks,
# )

model.evaluate(Xs, ys, verbose=1)

In [ ]:
prediction = model.predict(Xs)

In [ ]:
f,ax = plt.subplots(1,2,figsize=[14,7])

im = ax[0].imshow(prediction[:,:,1])
im = ax[1].imshow(ys.reshape(187,137))

f.colorbar(im)

In [ ]:


wandb.init(
    project="coralshift",
    entity="orlando-code",
    settings=wandb.Settings(start_method="fork")
    # config={    }
    )

# initialize optimiser: will need hyperparameter scan for learning rate and others
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
optimizer = tf.keras.optimizers.Adam(3e-4)

# X = ds_man.get_dataset("monthly_climate_1_12_X_np")
# y = ds_man.get_dataset("monthly_climate_1_12_y_np")
# # check that untrained model runs (should output array of non-nan values)
# # why values change?
# # g_model(X[:32])

# X_train, X_test, y_train, y_test = model_selection.train_test_split(
#     X, y, test_size=0.2, random_state=42)

# X_train, X_test, y_train, y_test = model_selection.train_test_split(
#     sub_X, sub_y, test_size=0.2, random_state=42)

# Define Gated Recurrent Unit model class in TensorFlow
class transformer_model(tf.keras.Model):
    # initialise class instance to define layers of the model
    # def __init__(self, rnn_units: list[int], num_layers: int):


   


    def call(self, inputs: np.ndarray, training: bool=False):
        """Processes an input sequence of data through several layers of GRU cells, followed by a couple of
        fully-connected dense layers, and outputs the probability of an event happening.
        
        Parameters
        ----------
        inputs (np.ndarray): input tensor of shape (batch_size, seq_length, features)
            batch_size - defines the size of the sample drawn from datapoints
            seq_length - number of timesteps in sequence
            features - number of features associated with each datapoint
        training (bool, defaults to False): True if model is in training, False if in inference mode

        Returns
        -------
        target: probability of an event occuring, with shape (batch_size, 1)
        """
        # input shape: (batch_size, seq_length, features)
       
        assert self.num_layers == len(self.rnn_units)

        # check that input tensor has correct shape
        if (len(inputs.shape) != 3):
            print(f"Incorrect shape of input tensor. Expected 3D array. Recieved {len(inputs.shape)}D array.")

        # print('input dim ({}, {}, {})'.format(inputs.shape[0], inputs.shape[1], inputs.shape[2]))
        # whole_seq, static_input = inputs
        whole_seq = inputs


        # iteratively passes input tensor to GRU layers, overwriting preceding sequence 'whole_seq'
        for layer_num in range(self.num_layers):
            whole_seq, final_s = self.grus[layer_num](whole_seq, training=training)

        # adding extra layers
        # static = self.B(tf.nn.gelu(self.A(static_input)))
        # target = self.w1(final_s)  + static # final hidden state of last layer used as input to fully connected dense layers...
        target = self.w1(final_s)   # final hidden state of last layer used as input to fully connected dense layers...

        target = tf.nn.relu(target) # via ReLU activation function
        target = self.w2(target)    # final hidden layer must have dimension 1 
        
        # obtain a probability value between 0 and 1
        target = tf.nn.sigmoid(target)
        
        return target


# initialise GRU model with 500 hidden layers, one GRU unit per layer 
g_model = gru_model([100], 1) # N.B. [x] is number of hidden layers in GRU network


def negative_log_likelihood(y: np.ndarray, y_pred: np.ndarray, class_weights: np.ndarray = None) -> float:
    """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
    incorporating class weights.

    Parameters
    ----------
    y (np.ndarray): true binary labels, where 0 represents the negative class
    y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)
    class_weights (np.ndarray): weights for each class. If None, no class weights will be applied.

    Returns
    -------
    float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred',
    incorporating class weights if provided
    """
    bce = tf.keras.losses.BinaryCrossentropy()  

    if class_weights is not None:
        sample_weights = tf.gather(class_weights, np.asarray(y,dtype=np.int32))
        # reshape to match size of y and y_pred
        return bce(y, y_pred, sample_weight=tf.reshape(sample_weights, (-1, 1)))

    return bce(y, y_pred)


def training_batches(X: np.ndarray, y: np.ndarray, batch_num: int, batch_size: int=32):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size

    X_batch = X[start_idx:end_idx]
    y_batch = y[start_idx:end_idx]
    
    return X_batch, y_batch

# https://stackoverflow.com/questions/52357542/attributeerror-tensor-object-has-no-attribute-numpy
# should aim to delete the following to speed up training: but can't figure out a way to make wandb reporting work
# without it
tf.config.run_functions_eagerly(True)

def build_graph():
    
    # compile function as graph using tf's autograph feature: leads to faster execution times, at expense of limitations
    # to Python objects/certain control flow structures (somewhat relaxed by experimental_relax_shapes)
    @tf.function(experimental_relax_shapes=True)
    def train_step(gru: tf.keras.Model, optimizer: tf.keras.optimizers.Optimizer, X: tf.Tensor, y: tf.Tensor, 
               num_batches: int, batch_num: int, training: bool = True, class_weights=class_weights
               ) -> tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
        """Train model using input `X` and target data `y` by computing gradients of the loss (via 
        negative_log_likelihood)
        
        Parameters
        ----------
        y (np.ndarray): true binary labels, where 0 represents the negative class
        y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)

        Returns
        -------
        float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred'
        """
        if training:

            num_batches = len(X)
            # num_samples = X.shape[0]
            # num_batches = num_samples // batch_size
            # num_batches = batch_num
            # total_epoch_loss = 0.0
            # for batch_num in tqdm(range(num_batches), desc="batches", position=0, leave=True):
            # for batch_num, batch in tqdm(enumerate(ds), desc="batches", position=0, leave=True):
            total_batch_loss = 0

            y_pred = gru(X, training)
            xent = negative_log_likelihood(y, y_pred, class_weights)

                # X_batch, y_batch = training_batches(X, y, batch_num=batch_num, batch_size=batch_size)

            with tf.GradientTape(persistent=True) as tape:
                y_pred = gru(X, training) 
                xent = negative_log_likelihood(y, y_pred, class_weights)
                # print(xent)
                # print(y_pred)
            
            gradients = tape.gradient(xent, gru.trainable_variables)
            optimizer.apply_gradients(zip(gradients, gru.trainable_variables))
            # print("xent", xent.numpy())
            # print("total_epoch_loss", total_epoch_loss)
            total_batch_loss += xent
                # learning rate?
            wandb.log({"batch": batch_num, "loss": xent, "total_epoch_loss": total_batch_loss})

            # average_loss = total_batch_loss / num_batches
            # return predicted output values and total loss value
            return y_pred, xent, total_batch_loss

    # set default float type
    tf.keras.backend.set_floatx('float32')
    return train_step


with tf.device("/GPU:1"):
    num_epochs = 20
    # will update so that subsamples are fed in from which batches are taken: will require recomputation
    # of class_weight for each subsample

    tr_step = build_graph()
    num_batches = len(train_dataset)

    for epoch in tqdm(range(num_epochs), desc= " epochs", position=1, leave=True):
        ongoing_epoch_loss = 0.0
        for i, (X_batch, y_batch) in tqdm(enumerate(train_dataset), position=0, total=len(train_dataset), desc=" training on batches", leave=True):
            y_pred, xent, batch_loss = tr_step(
                g_model, optimizer, X_batch, y_batch, class_weights=class_weights, 
                num_batches = num_batches, batch_num=i,
                training=True)

            ongoing_epoch_loss += batch_loss
            # print(ongoing_epoch_loss)
            wandb.log({"ongoing_epoch_loss": ongoing_epoch_loss})
        average_epoch_loss = ongoing_epoch_loss / len(train_dataset)
        # print(average_epoch_loss)
        wandb.log({"epoch": epoch, "tota_epoch_loss": average_epoch_loss})

wandb.finish()


In [ ]:
g_model.summary()

In [ ]:
pred = g_model(Xs, training=False)


In [ ]:
pred.shape

f,a = plt.subplots(figsize=[7,7])
im = plt.imshow(pred.numpy().reshape((187,137)))
f.colorbar(im)